# Prueba de bondad de ajuste

3. La siguiente muestra aleatoria corresponde a los rendimientos positivos de cierta acción a lo largo del tiempo:
0.2513, 0.2566, 0.3459, 0.6379, 2.0505, 1.803, 2.1906, 1.5299,0.35005, 0.3128, 1.2726, 2.3674, 2.3214, 2.4373, 0.6548.
## a) Prueba de Lilliefors
Se plantea que la muestra sigue una distribución normal. Realice la prueba de Lilliefors correspondiente para verificar esta suposición con un nivel de confianza del 90%

In [18]:
import pandas as pd
import numpy as np
from scipy import stats

# Paso 0: Realizar la prueba de Lilliefors
# Establecemos las hipotesis
# H0: La muestra sigue una distribución normal
# H0: La muestra No sigue una distribución normal

#Paso 1: Cargar los datos

df = pd.DataFrame({"rend": [ 0.2513, 0.2566, 0.3459, 0.6379, 2.0505, 1.803, 2.1906,
        1.5299, 0.35005, 0.3128, 1.2726, 2.3674, 2.3214, 2.4373, 0.6548 ]})

#Paso 2: Ordenar la muestra 

df = df.sort_values("rend").reset_index(drop=True)
x = df["rend"].values

#Paso 3: Construir la función empírica

def empirical (data):
    n = len(data)
    muestra = np.sort(data)
    empirical_values = []
    for v in muestra:
        empirical_values.append( np.sum(muestra <= v) /n)
    return empirical_values    

F_n = empirical(x)

# Paso 4: Calcular los parámetros de la función dada
n = len (x)
mu= np.mean(x) 
sigma2 = np.var(x, ddof=1)
sigma = np.sqrt(sigma2)

#Paso 5: Calcular la funcion de distribución acumulada bajo la distribución propuesta

F_x = stats.norm.cdf (x, loc=mu, scale=sigma)

lillie= pd.DataFrame({"x":x, "F(X)":F_x, "Fn(i)": F_n})

#Paso 6: Función desfasada

lillie["Fn(i-1)"] = lillie["Fn(i)"] - 1 / n

#Paso 7: Calcular D+ y D-
lillie["D+"] = abs(lillie ["F(X)"] - lillie ["Fn(i)"])
lillie["D-"] = abs(lillie ["F(X)"] - lillie ["Fn(i-1)"])

#Paso 7
D_mas = lillie["D+"].max()
D_menos = lillie["D-"].max()
#Paso 8 
Dn = max (D_mas, D_menos)
Dn

#Paso 9
#Para un nivel de significancia del 10%
alpha = 0.10
w_alpha = 0.2016
#Mostrar resultados
# 7. Mostrar todo
# -----------------------------
print("===== RESULTADOS LILLIEFORS =====\n")
print(f"Media estimada (μ): {mu:.5f}")
print(f"Desviación estándar estimada (σ): {sigma:.5f}")
print("\n--- Estadísticos ---")
print(f"D+     = {D_mas:.5f}")
print(f"D-     = {D_menos:.5f}")
print(f"Dn     = {Dn:.5f}")
print(f"Valor crítico w_alpha = {w_alpha:.5f}")

#Regla de decisión
if Dn > w_alpha:
    print("Rechazar Ho: Los datos no siguen una distribución normal")
else:
    print("No Rechazar Ho: Los datos siguen una distribución normal")
 

        

===== RESULTADOS LILLIEFORS =====

Media estimada (μ): 1.25214
Desviación estándar estimada (σ): 0.88379

--- Estadísticos ---
D+     = 0.21711
D-     = 0.15044
Dn     = 0.21711
Valor crítico w_alpha = 0.20160
Rechazar Ho: Los datos no siguen una distribución normal


# Conclusión
Como Dn= 0.21711  >  0.20160 = W_alpha se rechaza Ho. Hay evidencia suficiente para concluir que la nuestra No sigue una distribución normal con un nivel de significancia del 10%.

## a) Prueba de Kolmogorov–Smirnov
El gerente del banco asume que la muestra se distribuye según una distribución lognormal con media 0 y varianza 1. Realice la prueba de Kolmogorov Smirnov para verificar esta suposición con un nivel de significancia α = 0.01.

In [9]:
import pandas as pd
import numpy as np
from scipy import stats

# H0:X ~  LogN(01)
# H0: X no distribuye LogN(01)
# Datos  Cargar los datos
df = pd.DataFrame({"rend": [0.2513, 0.2566, 0.3459, 0.6379, 2.0505, 1.803, 2.1906,
        1.5299, 0.35005, 0.3128, 1.2726, 2.3674, 2.3214, 2.4373, 0.6548]})

# Ordenar
df = df.sort_values("rend").reset_index(drop=True)
x = df["rend"].values
n = len(x)

# Construir la funcion empirica
def empirical(data):
    n = len(data)
    muestra = np.sort(data)
    empirical_values = []
    for v in muestra:
        empirical_values.append(np.sum(muestra <= v) / n)
    return empirical_values

F_n = empirical(x)

# Lognormal(0,1)
# Si X ~ Lognormal(mu=0, sigma=1) entonces ln(X) ~ Normal(0,1)
F_x = stats.lognorm.cdf(x, s=1, scale=np.exp(0))

# DataFrame del K-S
ks_df = pd.DataFrame({"x": x, "F(X)": F_x, "Fn(i)": F_n})

# Fn(i-1)
ks_df["Fn(i-1)"] = ks_df["Fn(i)"] - 1/n

# D+ y D-
ks_df["D+"] = abs(ks_df["F(X)"] - ks_df["Fn(i)"])
ks_df["D-"] = abs(ks_df["F(X)"] - ks_df["Fn(i-1)"])

D_mas = ks_df["D+"].max()
D_menos = ks_df["D-"].max()

D_ks = max(D_mas, D_menos)

# Valor crítico del KS (α = 0.01)
alpha = 0.01
D_critico = 0.40420


# Impresiones
print(f"Estadístico de prueba KS = {D_ks:.5f}")
print(f"Valor crítico (α = 0.01) = {D_critico:.5f}")

# Decisión
if D_ks > D_critico:
    print("Rechazar H0: La muestra NO sigue una Lognormal(0,1)")
else:
    print("No Rechazar H0: No hay evidencia contra Lognormal(0,1)")

Estadístico de prueba KS = 0.18649
Valor crítico (α = 0.01) = 0.40420
No Rechazar H0: No hay evidencia contra Lognormal(0,1)


# Conclusión
D_ks < D_critico no se rechaza Ho. Hay evidencia suficiente para concluir que la nuestra sigue una distribución Lognormal(0,1) con un nivel de significancia del 1%.

5.  Una compañía de seguros analiza los montos de reclamos (en miles de dólares) por daños de  propiedad durante el último trimestre:8.4, 12.3, 15.7, 9.2, 21.4, 11.6, 13.8, 18.9, 10.5, 14.2,
16.8, 19.5, 22.1, 17.3, 20.6, 15.9, 23.4, 12.8, 16.4, 19.898
 El departamento actuarial necesita verifcon = 001

## a) Si los datos siguen una distribución exponencial mediante la prueba de Lilliefors con α = 
0.0 

In [14]:
import pandas as pd
import numpy as np
from scipy import stats

# Paso 0: Realizar la prueba de Lilliefors
# Establecemos las hipotesis
# H0: Los datos provienen de una distribución Exp(λ)
# H0: Los datos no provienen de una distribución Exp(λ)

#Paso 1: Cargar los datos

df = pd.DataFrame({"recl": [ 8.4, 12.3, 15.7, 9.2, 21.4, 11.6, 13.8, 18.9, 10.5, 14.2,
16.8, 19.5, 22.1, 17.3, 20.6, 15.9, 23.4, 12.8, 16.4, 19.8 ]})

#Paso 2: Ordenar la muestra 

df = df.sort_values("recl").reset_index(drop=True)
x = df["recl"].values

#Paso 3: Construir la función empírica

def empirical (data):
    n = len(data)
    muestra = np.sort(data)
    empirical_values = []
    for v in muestra:
        empirical_values.append( np.sum(muestra <= v) /n)
    return empirical_values    

F_n = empirical(x)

# Paso 4: Estimación del parámetro
n = len (x)
lambda_est = 1/ np.mean(x) 


#Paso 5: Calcular la funcion de distribución acumulada bajo la distribución propuesta

F_x = 1 - np.exp(-lambda_est * x)

#  Tabla de Lilliefors
lillie= pd.DataFrame({"x":x, "F(X)":F_x, "Fn(i)": F_n})
lillie["Fn(i-1)"] = lillie["Fn(i)"] - 1 / n

#Paso 7: Calcular D+ y D-
lillie["D+"] = abs(lillie ["Fn(i)"] - lillie ["F(X)"])
lillie["D-"] = abs(lillie ["F(X)"] - lillie ["Fn(i-1)"])


D_mas = lillie["D+"].max()
D_menos = lillie["D-"].max()
Dn_exp = max (D_mas, D_menos)



#Para un nivel de significancia del 5%
alpha = 0.05
w_alpha = 0.19939
#Mostrar resultados
# 7. Mostrar todo
# -----------------------------
print("===== RESULTADOS LILLIEFORS EXPONENCIAL =====\n")
print(f"estadístico (D): {Dn_exp:.5f}")
print(f"Valor crítico w_alpha = {w_alpha:.5f}")

#Regla de decisión
if Dn_exp > w_alpha:
    print("Rechazar Ho: Los datos no siguen una distribución exponencial")
else:
    print("No Rechazar Ho: Los datos siguen una distribución exponencial")
 

===== RESULTADOS LILLIEFORS EXPONENCIAL =====

estadístico (D): 0.40786
Valor crítico w_alpha = 0.19939
Rechazar Ho: Los datos no siguen una distribución exponencial


# Conclusión
Dn_exp > w_alpha, entonces se rechaza Ho. Hay evidencia suficiente para concluir que la nuestra no sigue una distribución exponencial con un nivel de significancia del 5%.

## b) Si los datos pueden modelarse mediante una distribución normal usando la prueba de Lilliefors α = 0.01

In [16]:
import pandas as pd
import numpy as np
from scipy import stats

# Paso 0: Realizar la prueba de Lilliefors
# Establecemos las hipotesis
# H0: La muestra sigue una distribución normal
# H0: La muestra No sigue una distribución normal

#Paso 1: Cargar los datos

df = pd.DataFrame({"recl": [ 8.4, 12.3, 15.7, 9.2, 21.4, 11.6, 13.8, 18.9, 10.5, 14.2,
16.8, 19.5, 22.1, 17.3, 20.6, 15.9, 23.4, 12.8, 16.4, 19.8 ]})

#Paso 2: Ordenar la muestra 

df = df.sort_values("recl").reset_index(drop=True)
x = df["recl"].values

#Paso 3: Construir la función empírica

def empirical (data):
    n = len(data)
    muestra = np.sort(data)
    empirical_values = []
    for v in muestra:
        empirical_values.append( np.sum(muestra <= v) /n)
    return empirical_values    

F_n = empirical(x)

# Paso 4: Calcular los parámetros de la función dada
n = len (x)
mu= np.mean(x) 
sigma2 = np.var(x, ddof=1)
sigma = np.sqrt(sigma2)

#Paso 5: Calcular la funcion de distribución acumulada bajo la distribución propuesta

F_x = stats.norm.cdf (x, loc=mu, scale=sigma)

lillie= pd.DataFrame({"x":x, "F(X)":F_x, "Fn(i)": F_n})

#Paso 6: Función desfasada

lillie["Fn(i-1)"] = lillie["Fn(i)"] - 1 / n

#Paso 7: Calcular D+ y D-
lillie["D+"] = abs(lillie ["F(X)"] - lillie ["Fn(i)"])
lillie["D-"] = abs(lillie ["F(X)"] - lillie ["Fn(i-1)"])

#Paso 7
D_mas = lillie["D+"].max()
D_menos = lillie["D-"].max()
#Paso 8 
Dn = max (D_mas, D_menos)


#Paso 9
#Para un nivel de significancia del 1%
alpha = 0.01
w_alpha = 0.2226
#Mostrar resultados
# 7. Mostrar todo
# -----------------------------
print("===== RESULTADOS LILLIEFORS =====\n")
print(f"Media estimada (μ): {mu:.5f}")
print(f"Desviación estándar estimada (σ): {sigma:.5f}")
print("\n--- Estadísticos ---")
print(f"D+     = {D_mas:.5f}")
print(f"D-     = {D_menos:.5f}")
print(f"Dn     = {Dn:.5f}")
print(f"Valor crítico w_alpha = {w_alpha:.5f}")

#Regla de decisión
if Dn > w_alpha:
    print("Rechazar Ho: Los datos no siguen una distribución normal")
else:
    print("No Rechazar Ho: Los datos siguen una distribución normal")
 


===== RESULTADOS LILLIEFORS =====

Media estimada (μ): 16.03000
Desviación estándar estimada (σ): 4.38647

--- Estadísticos ---
D+     = 0.06924
D-     = 0.09354
Dn     = 0.09354
Valor crítico w_alpha = 0.22260
No Rechazar Ho: Los datos siguen una distribución normal


# Conclusión
Como Dn = 0.09354 < 0.22260 = w_alpha, no hay evidencia suficiente para rechazar H0, por lo tanto los datos siguen una sistribuci[on normal con un nivel de significancia del 1%.
